When it comes to programming, there are many different way to solve a given problem. For instance, if we wanted to compute a math problem considering order of operations, we could write code like in test 1 below.

In [ ]:
#test 1
value1 = ((4 + 6)*2) / (2 + (9-3))
print(value1, " is the result.")
#end test 1

2.5  is the result.


The above code may be a solution, but it lacks the functionality of letting the user decide the math problem to compute. Let us attempt to print the result from user input in test 2

In [ ]:
#test 2
value2 = input("Give me an expression to evaluate : ")
print(value2, " is the result.")

Give me an expression to evaluate : 45+
45+  is the result.


So much for an interpretive language. Instead, we're going to solve this problem using data structures.

The following class is a Node.

In [ ]:
#Creates a Node
class Node:
    def __init__(self,data):                
        self.data=data                      
        self.next=None     
#end Node

Nodes are elements of data usually contained within data structures. The following data structure is a stack. 

In [ ]:
#Creates a Stack that has a Node as data        
class Stack:
    #Constructor
    def __init__(self):
        self.head=None                 #points head to top of stack
    #end Constructor

    #Assigns head to the new node
    def push(self,newNode):
        nextNode = newNode             #creates next node with passed in data
        nextNode.next = self.head      #links the node to Stack through the head
        self.head = nextNode           #assigns head to new top of stack
    #end push
 
    #Checks if stack is empty
    def isEmpty(self):
        if self.head is None :
          return True
        else :
          return False
    #Returns value at top of stack
    def peek(self):
      value = self.head.data
      return value
    #end isEmpty

    #Removes top of stack, assigns head to next node,and returns deleted value
    def pop(self):
        nodeToDelete = self.head      #assigns node to be deleted from head
        self.head = self.head.next    #shift head to the next available node
        value = nodeToDelete.data
        nodeToDelete.next = None      #deletes next data of node to delete
        nodeToDelete = None           #deletes node
        return value                  #returns value of deleted node if assigned
  #end pop
#end Stack

We can use structures like the stack in many useful ways. Below is an example of a stack being used to evaluate an expression in postfix form. (e.g. 45+)

In [ ]:
def evaluatePostfix(postfix):
  OperandStack = Stack()
  while postfix:                        #while string is not empty
    nextChar = postfix[0]               #assign first number in string to nextChar
    postfix = postfix[1:]               #removes number processed by nextChar
    #switch
    if nextChar == '+':
      rhs = OperandStack.pop()          #pops rhs operand
      lhs = OperandStack.pop()          #pops lhs operand
      addNode = Node(float(lhs) + float(rhs))
      OperandStack.push(addNode)        #inserts node as new operand
    elif nextChar == '-':
      rhs = OperandStack.pop()          
      lhs = OperandStack.pop()          
      subNode = Node(float(lhs) - float(rhs) )
      OperandStack.push(subNode)        #subtraction node as new operand
    elif nextChar == '*':
      rhs = OperandStack.pop()
      lhs = OperandStack.pop()           
      mulNode = Node(float(lhs) * float(rhs))
      OperandStack.push(mulNode)        #multiplication node as new operand
    elif nextChar == '/':
      rhs = OperandStack.pop()
      lhs = OperandStack.pop()          
      divNode = Node(float(lhs) / float(rhs))
      OperandStack.push(divNode)        #division node as new operand
    elif nextChar == '^':
      rhs = OperandStack.pop()
      lhs = OperandStack.pop()          
      expNode = Node(float(lhs) ** float(rhs))
      OperandStack.push(expNode)        #exponent node as new operand  
    else :
      numNode = Node(nextChar)
      OperandStack.push(numNode)
    #end switch
  #end while
  return OperandStack.pop()            #pops top node, which is result
#end evaluatePostfix

Test 3 below uses the function we just implemented to evaluate the postfix expression passed in by the user.

In [ ]:
       
#test 3
postfix = input("Give me a postfix expression to evaluate : ")
value3 = evaluatePostfix(postfix)
print(value3, "is the result.")
#end test 3

Give me a postfix expression to evaluate : 45+
9.0 is the result.


This solution still isn't optimal. If we want the user to define the expression as done normally through infix, delimited expressions, we need to add functionality and move more data around. First let's program a couple functions to help check for delimiters.

In [ ]:
#Function helper that checks delimiter pairing
def isPaired(open, close):
  #returns True if paired
  return open == '(' and close == ')'
#end isPaired

#Checks if delimiters are balanced
def delimitersOk(expression):
  #function variables      
  inputline = expression
  DelimiterStack = Stack()
  index = 0
  isBalanced = True
  characterCount = len(inputline)
  while isBalanced and (index < characterCount) : #stay in loop 'til unbalanced
                                                  #or expression traversed
    nextCharacter = inputline[index]
    if nextCharacter == '(' :
      delimNode = Node(nextCharacter)
      DelimiterStack.push(delimNode)              #push character if delimiter
    elif nextCharacter == ')' :
      if DelimiterStack.isEmpty() :
        isBalanced = False                        #unbalanced if stack is empty
      else :
        openDelimiter = DelimiterStack.pop()      #pop delimiter
        isBalanced = isPaired(openDelimiter, nextCharacter)
      #end if
    else :
      pass
    #end if
    index += 1
  #end while
  if DelimiterStack.isEmpty() == False :
    isBalanced = False
  return isBalanced
#end delimitersOk

Let's do a sanity check to see that the function is working properly.

In [ ]:
validExp = "((4 + 6)*2) / (2 + (9-3))"
invalidExp = "((4 + 6)*2) / (2 + (9-3)"
if delimitersOk(validExp) == True :
  print("((4 + 6)*2) / (2 + (9-3)) is valid.")
if delimitersOk(invalidExp) == False :
  print("((4 + 6)*2) / (2 + (9-3) is invalid.")


((4 + 6)*2) / (2 + (9-3)) is valid.
((4 + 6)*2) / (2 + (9-3) is invalid.


Now, we need to parse our information, and we are set to allow user input for math problems requiring order of operations.

In [ ]:
def process(nextChar, postfix, OperatorStack):
  #switch
  if ord(nextChar) > 47 and ord(nextChar) < 58 :  #if next character is operand
      postfix = postfix + nextChar
  elif nextChar == '^' :
    expNode = Node(nextChar)
    OperatorStack.push(expNode)
  elif nextChar == '(' :
    delimNode = Node(nextChar)
    OperatorStack.push(delimNode)
  elif nextChar == ')' :
    topOperator = OperatorStack.pop()
    while topOperator != '(' :
      postfix = postfix + topOperator
      topOperator = OperatorStack.pop()
    #end while
  elif nextChar == '+' or nextChar == "-" or nextChar == "*" or nextChar == "/" :
      while OperatorStack.isEmpty() == False and nextChar <= OperatorStack.peek() :
        topOperator = OperatorStack.pop()
        postfix = postfix + topOperator
        topOperator = OperatorStack.pop()
      #end while
      opNode = Node(nextChar)
      OperatorStack.push(opNode)
  else :
    pass
  #end switch
  return postfix
#end process

def toPostfix(infix):
  index = 0
  characterCount = len(infix)
  OperatorStack = Stack()
  postfix = ""
  while index < characterCount :
    nextChar = infix[index]
    postfix = process(nextChar, postfix, OperatorStack)
    index += 1
  #end while
  while OperatorStack.isEmpty() == False :
    topOperator = OperatorStack.pop()
    postfix = postfix + topOperator
  #end while
  return postfix
#end toPostfix

In [ ]:
#test 4
expression = input("Enter a delimited expression 'i.e. (4 + 6) / (2 + 3)' to evaluate: ")
valid = delimitersOk(expression)
if valid :
  postfix = toPostfix(expression)
  value4 = evaluatePostfix(postfix)
  print(value4, "is the result.")
#end if
#end test 4

Enter a delimited expression 'i.e. (4 + 6) / (2 + 3)' to evaluate: 4+6/2+3
462/3+ is the postfix form.
6.0 is the result.


There we have it! A python program that will attempt solve an order of opertations problem. Of course there are still bugs that causes errors for some inputs, but this is a good starting point to having a program that allows the user to input the math problem.
